# Artistic Style Transfer 

This Colab contains a replication of the artistic style transfer proposed by Leon A. Gatys, Alexander S. Ecker and Matthias Bethge in their paper 'A Neural Algorithm of Artistic Style' (2015).

*Replication by Marisa Wodrich and Johanna Linkemeyer*

---

## **1 Introduction**

Artistic style transfer aims at transferring the style of a given image to an artistic style. It allows to take any photopraph and any artistic work one likes and creates a version of that photograph in the style (meaning the same colors, brushstroke, and art specific patterns) of the specific artwork.

In 2015, Gatys and colleagues proposed a deep neural network (DNN) to perform artistic style transform on any given image. Their approach is based on the following finding: Visualizing the reconstructions of images that are processed with the object recognition network VGG, the researchers found that the image content gets increasingly precise along the layers of the convolutional neural network. At the same time, they found that style can be reconstructed using the correlation of the feature maps in each layer. Gatys and colleagues used this finding to generate new images that contain the content of a content image and the style of an artistic style image. They do so by iteratively adapting the generated image by applying the obtained gradients to the generated image. Those gradients get smaller and the image gets adapted less, the more the content of the generated image matches that of the original content image while also precisely capturing the style of the original style image. The overall loss to calculate the gardients in each iteration is computed using the sum of a content and a style loss. The relation of those two losses can be adapted with a content weight and a style weight - which in the end determine the how much content and style are represented in the final, generated image. The following image depicts the generated result (bottom) of the artistic style transfer from the following code using the example of a sunflower image (top) and the style of the artist James Rizzy (middle).

<img src='https://www.myhomebook.de/data/uploads/2020/02/gettyimages-1141659565-1024x683.jpg' width='200px'>
<img src='https://github.com/marisawodrich/style-transfer/blob/main/figures/rizzy.jpg?raw=true' width='200px'>
<img src='https://github.com/marisawodrich/style-transfer/blob/main/figures/sunflower-rizzy.jpg?raw=true' width='200px'>

## **2 Data**

In this section, the data for our style transfer is loaded. Also, the functions to preprocess our data for the network are defined here. The data we use are images. More precisely, for our artistic style transfer, we require content and style images. Content images can be basically any image. We chose the Neckarfront image from the paper we replicate and, additionally, some other images we found on the internet to test the effectiveness of the style transfer on other images, too:
* Image of the Neckarfront in Tübingen (Original paper by Gatys and colleagues)
* A Field of Sunflowers
* A Puppy on a Field
* A Lake in Front of a Mountain

For the style images, we chose the five style images from the original paper and added two more that we liked particularly. The style images provided from us are:
* The Starry Night by Vincent van Gogh (1889) *- Post-Impressionism*
* The Scream by Edvard Munch (1893) *- Expressionism*
* Cafeterrasse am Abend by Vincent van Gogh (1888) *- Post-Impressionism*
* Style from the Artist James Rizzy *-* 
* The Shipwreck of the Minotaur by William Turner (1810) *- Romantic*
* Composition VII by Wassily Kadinsky (1913) *- Abstract Art*
* Femme Nue Assise by Pablo Picasso (1910) *- Cubism*


### **2.1 Setup**

This section is for setup purposes. All necessary packages are imported and the required directories to store the data are created.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, Model

import numpy as np
import os 
import matplotlib.pyplot as plt
import time
import functools
import cv2
from skimage import io
from PIL import Image
from IPython.display import clear_output

In [ ]:
from helpers import make_dirs

In [ ]:
# Define paths directories for images (subdivided into style, content and 
# generated images) if they do not exist.
path = './images'
path_style = os.path.join(path, 'style')
path_content = os.path.join(path, 'content')
path_generated = os.path.join(path, 'generated')

# Create directories from predefined paths.
make_dirs([path, path_style, path_content, path_generated])

### **2.2 Load and Visualize Images**

The following sections contains all functions to load images and visualize them.

There is the option to add an image from your own link or your computer if you have a preferred image for style transfer. You can find the instructions to use your own image in the section **2.1.2 Load Images**.



#### **2.2.1 Helper Functions**

In [ ]:
from helpers import update_dictionary, test_jpg_or_png, load_image_from_link

#### **2.2.2 Load Images**

Here, the functions above are used to load some images that can/ will later be used for style transfer. We provide seven style images and four content images. The images from the paper by Gatys and colleagues are included to allow a comparison of our results to theirs.

If you would like to use a style or content image of your choice from a link, feel free to use the 

>**load_image_from_link(link, image_name, image_description, style_image)** 

function. For the parameters, please adapt them to your needs in the following way:

* **link**: Here, you should insert the link to your image. Please use the link that explicitly leads to the image, not the website you obtain it from. To check, this link should end with **.jpg** or **.png**. For some images, even though you provide the correct link, this method of loading the image won't work. You can still use the image by saving it on your device and uploading it manually to the respective folder.

* **image_name**: Here, you should insert the name you want to give the image. Please check, that the name ends with **.jpg** or **.png**.

* **image_description**: Please provide a short description of the image. This description will be used as a title in the image visualization.

* **style_image**: Depending on whether you would like to add a new style or content image, set this boolean to *true* or *false*. Choose *true* if you add a style image, and *false* for a content image.



In [ ]:
# ---- STYLE IMAGES ----

# The Starry Night by Vincent van Gogh (1889)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1280px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg',
                     'starry_night.jpg',
                     'Starry Night',
                     style_image=True)

# The Scream by Edvard Munch (1893)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg/300px-Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg',
                     'the_scream.jpg',
                     'The Scream',
                     style_image=True)

# Cafeterrasse am Abend by Vincent van Gogh (1888)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Gogh4.jpg/300px-Gogh4.jpg',
                     'cafeterrasse_am_abend.jpg',
                     'Caféterrasse am Abend',
                     style_image=True)

# Style from the Artist James Rizzy
load_image_from_link('http://www.james-rizzi.com/wp-content/uploads/pictures/cache/2007_03_000_KeepingBusyInARizziCity_800_600.jpg',
                     'rizzy.jpg',
                     'Rizzy Style',
                     style_image=True)

# The Shipwreck of the Minotaur by William Turner (1810)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/2/2e/Shipwreck_turner.jpg',
                     'shipwreck.jpg',
                     'The Shipwreck of the Minotaur',
                     style_image=True)

# Composition VII by Wassily Kadinsky (1913)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg',
                     'composition_vii.jpg',
                     'Composition VII',
                     style_image=True)

# Femme Nue Assise by Pablo Picasso (1910)
load_image_from_link('https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg',
                     'femme_nue_assise.jpg',
                     'Femme nue assise',
                     style_image=True)

# ---- CONTENT IMAGES ----

# Image of a Field of Sunflowers
load_image_from_link('https://www.myhomebook.de/data/uploads/2020/02/gettyimages-1141659565-1024x683.jpg',
                     'sunflower.jpg',
                     'Sunflower',
                     style_image=False)

# Image of a Lake in Front of a Mountain
load_image_from_link('https://www.scinexx.de/wp-content/uploads/0/1/01-35131-nukliduhr01.jpg',
                     'mountain.jpg',
                     'Mountain',
                     style_image=False)

# Image of a Puppy on a Field
load_image_from_link('https://www.mera-petfood.com/files/_processed_/a/4/csm_iStock-521697453_7570f7a9b6.jpg',
                     'puppy.jpg',
                     'Puppy',
                     style_image=False)

# Image of the Neckarfront in Tübingen
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg',
                     'neckarfront.jpg',
                     'Neckarfront',
                     style_image=False)


#### **2.2.3 Image Visualization**

Here, one can take a look at the loaded data. Running the following two code blocks will visualize all style and content images that are in the respective folder.

In [ ]:
from visualize import visualize_images

In [ ]:
visualize_images(path_style, 'STYLE IMAGES', notebook=True)
visualize_images(path_content, 'CONTENT IMAGES', notebook=True)

### **2.3 Preprocessing Functions**

For this project, we will use pre-trained layers from VGG19, a Convolutional Neural Network (CNN) for object detection. Input images to VGG19 must be preprocessed in the following way:

* Size of images: 224 x 224 x 3
* Color range: [0, 255]
* Value type: float 32
* Colorspace: BGR
* Normalization (zero-centering) per channel (B: 103.939, G: 116.779, R: 123.68)

*This information is obtained from [here](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg19/preprocess_input) and [here](https://towardsdatascience.com/extract-features-visualize-filters-and-feature-maps-in-vgg16-and-vgg19-cnn-models-d2da6333edd0) and not provided in the paper by Gatys and colleagues.*

NOTE: We do not process all data right away because we can do it easily when running the style transfer later.

In [ ]:
from processing import img2tensor, tensor2img

Testing the preprocessing pipeline:

In [ ]:
# Define path to test image
img_path = os.path.join(path_content, 'puppy.jpg')

# Preprocessing of given image
img_tensor, orig_shape = img2tensor(img_path)

# Test if image actually is a tensor
assert tf.is_tensor(img_tensor), 'Image is not a tensor.'

# Test if conversion back works as well
img = tensor2img(img_tensor, orig_shape)

# Show image to check if transfer back worked.
plt.imshow(img)
plt.axis('off')
plt.show()

## **3 Model**

In the orginal the VGG-network is used. VGG is a Convolutional Neural Network (CNN) for object detection. We will first inspect the original model and then define our own model based on the pretrained layers (imagenet) of the VGG19 network.

### **3.1 Inspect VGG19**

A pretrained version of VGG19 is available from the keras applications. We load the model with the imagenet weights. We also specify that we would like to use average pooling (indicated by the parameter *pooling='avg'*) because this variant is used in the original paper. Another variant would be *pooling='max'* for max-pooling.

In [ ]:
# Load the model
model_vgg19 = tf.keras.applications.VGG19(include_top=False,
                                          weights='imagenet',
                                          pooling='avg')

# Inspect the layers.
for layer in model_vgg19.layers:
    print(layer.name)

We can observe that the model consists of an input layer, followed by five blocks and finished off by a global average pooling layer. Each block consists of four convolutional layers, finished with a pooling layer.

### **3.2 Get Model**

In [ ]:
# Import the model class that can be defined with specific layers from the pretrained VGG network
from model import StyleModel

### **3.3 Define Gram Matrix and Loss Functions**

The style transfer is achieved through a very specific loss. The overall loss consists of the sum of weighted content and style losses. The formula for the overall loss is as follows:
<center>$L_{total}(\vec{p}, \vec{a}, \vec{x}) = \alpha * L_{content}(\vec{p}, \vec{x}) + \beta * L_{style}(\vec{a}, \vec{x})$</center>

with:
* $\vec{p} = $ content image/ photograph
* $\vec{a} = $ artwork image
* $\vec{x} = $ generated image
* $\alpha = $ weight for the content loss
* $\beta = $ weight for the style loss


$\alpha$ and $\beta$ will be provided by the user when running the style transfer. In the following sections, there will be a detailed description on how exactly the content and style losses are calculated, respectively.

In [ ]:
# This line imports all functions to calculate the specific content (3.3.3) and style loss (3.3.2) and the 
# gram matrices (3.3.1) for each layer to get the style loss. The documentation for the calculations done 
# in the functions can be found in the following subsections (3.3.1 - 3.3.3)

from model import gram_matrix, loss_style, loss_content

#### **3.3.1 Gram Matrix**

The Gram Matrix is necessary for the calculation of the style loss. The idea of the gram matrix is to get the correlation of the filter responses for each layer. Those feature correlations are provided through the inner product of the vectorized feature maps (for each layer). So, the gram matrix has a shape of $(number\_channels * number\_channels)$.

The formula for the gram matrix is as follows:
<center>$G_{ij}^l = \sum_k F_{ik}^l * F_{jk}^l$</center>

with:
* $l = $ current layer
* $G^l = $ gram matrix in layer $l$
* $F^l = $ feature map in layer $l$

#### **3.3.2 Style loss**

The style loss is the mean squared error of the gram matrices of output and target for each layer Outputs and targets are the outputs from the model of the generated image (output) and the original style image (target) for all relevant style layers.

The loss is then computed by iterating over the outputs and targets for each style layer and calculating the error for those layers respectively. Each 'layer-error' is then weighted by the number of style layers.

To get the difference for original and style images' outputs in the specific layer, the gram matrices are calculated for both. The sum of squared difference is then weighted to get the mean. The full formula is as follows.

<center>$L_{style}(\vec{a}, \vec{x}) = \sum_{l=0}^L w_l * E_l$</center>

with
* $\vec{a} = $ original image/ artwork image
* $\vec{x} = $ generated image
* $w = $ weighting factor 1 / number of style layers
* and

<center>$E_l = \dfrac{1}{4 * N_l^2 * M_l^2} * \sum_{i, j} (G_{ij}^l - A_{ij}^l)^2$</center>

with
* $L =$ all layers
* $N_l =$ number of feature maps in layer $l$
* $M_l =$ size of feature maps (its height * its width) in layer $l$
* $A^l =$ gram matrix of target image in layer $l$
* $G^l =$ gram matrix of generated image in layer $l$


#### **3.3.3 Content Loss**

The content loss is the mean squared error over the feature representations of the output and target per layer.

<center>$L_{content}(\vec{p}, \vec{x}, l) = \frac{1}{2} * \sum_{i,j} (F_{ij}^l - P_{ij}^l)^2$</center>

with
* $\vec{p} = $ content image/ photograph
* $\vec{x} = $ generated image
* $l = $ current layer $l$
* $P^l = $ feature representation in layer $l$ of content image
* $F^l = $ feature representation in layer $l$ of generated image


## **4 Running the Style Transfer**

In style transfer, there is no real 'training' phase but rather a phase where the content image iteratively gets adapted such that it takes the style of the style image. First

### **4.1 Helper Functions**

In [ ]:
# Import helper functions and code for visualization of progress
from helpers import prep_images, clip_values
from visualize import visualize_progress

### **4.2 Define Model with Specific Layers and Default Parameters**

For the content layers of our model for artistic style transfer, we chose 'block4_conv2' as in the original paper.

For the style layers, we chose the variant from the original paper. From each block, the first convolutional layer is used in the style transfer model.

In [ ]:
# Content layers: Second convolutional layer from the forth block.
layers_content = ['block4_conv2'] 

# Style layers: First convolutional layer of each of the five blocks
layers_style = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

assert len(layers_content) != 0, 'There must be at least one content layer.'
assert len(layers_style) != 0, 'There must be at least one style layer.'

# Append the layer names to define the full model.
layers_complete = layers_content + layers_style

In [ ]:
# Set model with the previously defined layers
model = StyleModel(number_content_layers=len(layers_content), layer_names=layers_complete)

# Model should not be trainable
for layer in model.layers:
    layer.trainable = False

# --- Default Parameters ---

# Optimizer
optimizer = tf.optimizers.Adam(learning_rate=4)

# Weights
weight_content_default = 1
weight_style_default = 1e-2

# Number of iterations
iterations_default = 2000

# Image names
image_content_name = 'neckarfront.jpg'
image_style_name = 'composition_vii.jpg'
save_name_default = image_content_name.split('.')[0] + '-' + image_style_name

# Load images
image_content_default, image_style_default, image_content_default_shape = \
    prep_images(image_content_name, image_style_name)

### **4.3 Style Transfer Function**

In [ ]:
from main import transfer_artistic_style

In [ ]:
img_names = transfer_artistic_style(model,
                                    iterations_default, 
                                    image_content_default, 
                                    image_style_default, 
                                    optimizer, 
                                    weight_content_default,
                                    weight_style_default,
                                    image_content_default_shape, 
                                    save_name_default,
                                    visualize_intermediate=True)

In [ ]:
# Run visualization code
visualize_progress(img_names)

## **5 Results**

To get all results, we first run the style transfer on all possible combinations on the content images we chose and then on all combinations from the original paper. The names of the result images are stored and can later be visualized.

In [ ]:
from main import run_all_combinations
from visualize import plot_combinations

### **5.1 Results on Selected Content Images**

Here are the results of all possible combinations of the puppy, sunflower, and mountain content images with the style images: the scream, the starry night, Caféterrasse am Abend, and James Rizzy style.

The first cell will run the code for all possible combinations and the second cell will visualize the original content and style image and the generated image next to each other for comparison.

In [ ]:
# Define the our content image names and the the style images we would like
content_images = ['puppy.jpg', 'sunflower.jpg', 'mountain.jpg']
style_images = ['the_scream.jpg', 
                'cafeterrasse_am_abend.jpg', 
                'starry_night.jpg', 
                'rizzy.jpg']

print('--- PROGRESS START ---')

# Run style transfer on all those possible combinations
save_names_own_content_imgs = run_all_combinations(model,
                                                   content_images, 
                                                   style_images,
                                                   optimizer,
                                                   weight_content_default,
                                                   weight_style_default,
                                                   iterations_default)

print('--- PROGRESS FINISHED, GOT ALL COMBINATIONS ---')

In [ ]:
# Visualization
plot_combinations(save_names_own_content_imgs)

### **5.2 Results on Original Combinations**

In [ ]:
# Define content and style images from original paper
content_images = ['neckarfront.jpg']
style_images = ['starry_night.jpg', 
                'composition_vii.jpg', 
                'the_scream.jpg', 
                'femme_nue_assise.jpg', 
                'shipwreck.jpg']

print('--- PROGRESS START ---')

# Run style transfer on all combinations from the original paper
save_names_original_paper = run_all_combinations(model,
                                                 content_images,
                                                 style_images,
                                                 optimizer,
                                                 weight_content_default,
                                                 weight_style_default,
                                                 iterations_default)

print('--- PROGRESS FINISHED, GOT ALL COMBINATIONS ---')

In [ ]:
plot_combinations(save_names_original_paper)

## **6 Comparison to Original Paper**

Please refer to our paper [here](https://github.com/marisawodrich/style-transfer/blob/main/replication_paper/style_transfer_not_finished.pdf) for a thorough analysis of our results and analyses and a comparison of those to the orgininal paper by Gatys and colleagues.

## **References**

* Gatys, L. A., Ecker, A. S., & Bethge, M. (2015). A neural algorithm of artistic style. arXiv preprint arXiv:1508.06576.

* Neural Style Transfer with tf.keras. https://colab.research.google.com/github/tensorflow/models/blob/master/research/nst_blogpost/4_Neural_Style_Transfer_with_Eager_Execution.ipynb#scrollTo=sElaeNX-4Vnc [accessed 2021-04-03].

* TensorFlow (last Update: 2021). Neural style transfer. https://www.tensorflow.org/tutorials/generative/style_transfer?hl=en [accessed 2021-04-04].

* Extract Features, Visualize Filters and Feature Maps in VGG16 and VGG19 CNN Models (2020). https://towardsdatascience.com/extract-features-visualize-filters-and-feature-maps-in-vgg16-and-vgg19-cnn-models-d2da6333edd0 [accessed 2021-04-04].

### **Images**
* Image of the Neckarfront in Tübingen. https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg [accessed 2021-04-04].

* A Field of Sunflowers. https://www.myhomebook.de/data/uploads/2020/02/gettyimages-1141659565-1024x683.jpg [accessed 2021-04-04].

* A Puppy on a Field. https://www.mera-petfood.com/files/_processed_/a/4/csm_iStock-521697453_7570f7a9b6.jpg [accessed 2021-04-04].

* A Lake in Front of a Mountain. https://www.scinexx.de/wp-content/uploads/0/1/01-35131-nukliduhr01.jpg [accessed 2021-04-04].

* Vincent van Gogh (1889). The Starry Night. https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1280px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg[accessed 2021-04-04].

* Edvard Munch (1893). The Scream. https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg/300px-Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg [accessed 2021-04-04].

* Vincent van Gogh (1888) Cafeterrasse am Abend. https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Gogh4.jpg/300px-Gogh4.jpg [accessed 2021-04-04].

* Style from the Artist James Rizzy. http://www.james-rizzi.com/wp-content/uploads/pictures/cache/2007_03_000_KeepingBusyInARizziCity_800_600.jpg [accessed 2021-04-04].

* William Turner (1810). The Shipwreck of the Minotaur. https://upload.wikimedia.org/wikipedia/commons/2/2e/Shipwreck_turner.jpg [accessed 2021-04-04].

* Wassily Kadinsky (1913). Composition VII. https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg [accessed 2021-04-04].

* Pablo Picasso (1910). Femme Nue Assise. https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg [accessed 2021-04-04].